In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import regex as re
import time
import functools as f

In [12]:
def counter(df, label):
  word_count = {}
  for cat in categories:
    word_dict = {}
    data = df[df[label] == cat]['headline'] +' '+ df[df[label] == cat]['short_description']
    for row in data:
      words = list(set(re.split("[^a-zA-Z]",row)))#.split(' ')
      for word in words:
        word = word.lower()
        if word in word_dict:
          word_dict[word] = word_dict[word]+1
        else:
          word_dict[word] = 1
    for word, count in dict(word_dict).items():
      if (len(word)<3) or (count<4):
        word_dict.pop(word)
    word_count[cat] = word_dict
  return pd.DataFrame(word_count).fillna(0)

In [13]:
def prob_cond(word, category, word_df, a=0):
  """word: word whose probability needs to be found
     category: prior condition of the probabilty
     word_df: dictionary of the count of word in categories
     a: laplace smoothing factor"""
  word = word.lower()
  category = category.upper()
  if word in vocab:
    prob = (word_df.loc[word,category]+a)/(word_df[category].sum()+a*len(word_df[category]))
  else:
    prob = (a)/(word_df[category].sum()+a*len(word_df[category]))
  return prob

In [14]:
def prob(word, prob_df):
  word = word.lower()
  probblty = 0
  try:
    for cat in prob_df.columns:
      probblty += prob_df.loc[word,cat]*p_cat[cat]
    return probblty
  except:
    return probblty

In [15]:
def train(df,label='category', smoothing = 1):
  global categories
  categories = df[label].unique()
  temp = df[label].value_counts(sort=False)/df[label].value_counts().sum()
  p_cat = temp.to_dict()

  word_count = counter(df, label)
  global vocab
  vocab = word_count.index
  pc_word = {}
  for cat in categories:
    word_prob = {}
    for word in vocab:#word_count[cat].keys():
      word_prob[word] = prob_cond(word,cat,word_count,a=smoothing)
    pc_word[cat] = word_prob
  pc_word = pd.DataFrame(pc_word)
  return p_cat, word_count, pc_word

In [16]:
def predict(X):
  cat_pred = np.array([])
  #data = X#df_val['headline']#df_val['short_description']+' '+df_val['headline']
  for row in X:
    val_words = list(set(re.split("[^a-zA-Z]",row)))
    for word in val_words:
      if (len(word)<3):
        val_words.remove(word)
    cat_prob = np.array([])
    for cat in categories:
      list_prob = np.array([])
      for word in val_words:
        if word in vocab:#try:
          list_prob = np.append(list_prob,pc_word.loc[word,cat])#,word_count,a=1))
        else:
          list_prob = np.append(list_prob,prob_cond(word,cat,word_count,a=smoothing))
      cat_prob = np.append(cat_prob,np.prod(list_prob)*p_cat[cat])
    cat_prob = cat_prob/sum(cat_prob)
    # id = np.argmax(cat_prob)
    cat_pred = np.append(cat_pred,np.argmax(cat_prob))
  return cat_pred

In [17]:
def accuracy(prediction, actual):
  accu = 0
  for i in range(len(prediction)):
    if prediction[i] == actual[i]:
      accu += 1
  accu = accu/len(prediction)
  return accu

In [18]:
#!wget -O news.json 'https://drive.google.com/uc?export=download&id=1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp'
# # https://drive.google.com/file/d/1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp/view?usp=sharing

In [19]:
df = pd.read_json('news.json',lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [20]:
df['category'] = df['category'].replace(['CULTURE & ARTS','THE WORLDPOST','PARENTS','ARTS','STYLE','TASTE','WELLNESS','WORLDPOST'],
                                        ['ARTS & CULTURE','WORLD NEWS','PARENTING','ARTS & CULTURE','STYLE & BEAUTY','FOOD & DRINK','HEALTHY LIVING','WORLD NEWS'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           200853 non-null  object        
 1   headline           200853 non-null  object        
 2   authors            200853 non-null  object        
 3   link               200853 non-null  object        
 4   short_description  200853 non-null  object        
 5   date               200853 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [22]:
df, df_test = train_test_split(df, test_size=0.1, stratify=df['category'])
# df_val, df_test = train_test_split(df_val, test_size=0.5, random_state=42)

In [23]:
smoothing = 1

In [24]:
# p_cat, word_count, pc_word = train(df, 'category', stpwrds=stpwrds, smoothing=smoothing)
# cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}


In [25]:
# start = time.time()
# input = df_val['headline']#+' '+df_val['short_description']
# predictions = predict(input)
# predictions = [cat_dict[v] for v in predictions]
# print(time.time()-start)

In [26]:
cv= 5
split_df = np.array_split(df.copy(), cv)
score = []
for i in range(cv):
    df_val = split_df[i]
    diff = split_df[:i]+split_df[i+1:]
    df_train = f.reduce(lambda a, b: pd.concat([a,b]),diff)
#    df_train, df_val = train_test_split(df, test_size=0.1)#, random_state=42)
    p_cat, word_count, pc_word = train(df_train, 'category', smoothing=smoothing)
    cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}
    input = df_val['headline']#+' '+df_val['short_description']
    predictions = predict(input)
    predictions = [cat_dict[v] for v in predictions]
    actual = df_val['category'].values
    score.append(accuracy(predictions,actual))
print('Cross validation scores are: ', score)
print('Cross validation mean score is: ', np.mean(score))

Cross validation scores are:  [0.5920506721247994, 0.5886485589423024, 0.5906840372859791, 0.5857881780211878, 0.5895499681907449]
Cross validation mean score is:  0.5893442829130027


In [27]:
p_cat, word_count, pc_word = train(df, 'category',smoothing=smoothing)
cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}
print("Probability of 'The' occuring in a headline is: ", round(prob('The',pc_word),3))
print("Probability of 'The' occuring in a 'Business' headline is: ", round(prob_cond('The'.lower(),'BUSINESS'.upper(),word_count, a=smoothing),3))

Probability of 'The' occuring in a headline is:  0.037
Probability of 'The' occuring in a 'Business' headline is:  0.038


In [28]:
input = df_test['headline']+' '+df_test['short_description']
predictions = predict(input)
predictions = [cat_dict[v] for v in predictions]
actual = df_test['category'].values
accuracy(predictions,actual)

C:\Users\TKC2CC~1.-20\AppData\Local\Temp/ipykernel_16540/2667250092.py:18: RuntimeWarning: invalid value encountered in true_divide
  cat_prob = cat_prob/sum(cat_prob)


0.6094792392711341

In [29]:
full_prob = []
for cat in categories:
  word_prob = []
  for word in vocab:
    word_prob.append(pc_word.loc[word,cat]*p_cat[cat]/prob(word,pc_word))
  full_prob.append(word_prob)
full_prob = np.array(full_prob)
full_prob_df = pd.DataFrame(full_prob.T,columns=categories,index=vocab)

top_word = []
for cat in categories:
  top_word.append(full_prob_df[cat].sort_values(ascending=False).index)
top_word = np.array(top_word)
top_words_df = pd.DataFrame(top_word.T,columns=categories)


In [30]:
top_words_df.iloc[:10,:16]

,BLACK VOICES,HEALTHY LIVING,POLITICS,TECH,WORLD NEWS,HOME & LIVING,GREEN,WEDDINGS,QUEER VOICES,SPORTS,DIVORCE,STYLE & BEAUTY,FOOD & DRINK,TRAVEL,COMEDY,PARENTING
0,panther,gps,democrats,samsung,migrants,decor,wildfire,weddings,queer,ncaa,divorce,huffpoststyle,recipes,destinations,maher,breastfeeding
1,trayvon,workout,huffpollster,iphone,aleppo,homesubmissions,meatless,brides,lgbtq,nba,divorced,fashion,recipe,hotels,colbert,parenting
2,mlk,diabetes,sen,microsoft,rohingya,furniture,carbon,bride,lesbian,touchdown,divorcing,stylelist,cocktails,traveler,meyers,mommy
3,blacks,workouts,democrat,unconfirmed,mosul,pitches,elephants,wedding,bisexual,tournament,infidelity,middleton,baked,travelers,conan,toddlers
4,philando,fitness,senate,android,referendum,crafts,oceans,groom,gay,lebron,blended,tumblr,dessert,attractions,trevor,toddler
5,black,strain,republicans,apple,rebels,repurposing,seaworld,bridal,trans,mlb,cheating,wardrobe,salad,resorts,spoof,motherhood
6,luther,mindfulness,rubio,google,myanmar,decorating,conservation,grooms,lgbt,nfl,breakup,outfit,sauce,beaches,seth,darndest
7,castile,exercise,marco,ios,ceasefire,sprout,fracking,nuptials,rupaul,championship,divorces,outfits,desserts,tsa,fallon,parent
8,duvernay,fearless,mcconnell,iphones,minister,diy,wildfires,planner,transgender,quarterback,alimony,makeup,cheese,getaways,kimmel,moms
9,hbcu,chronic,senator,encryption,militants,appraises,pollution,knot,homophobia,playoff,exes,runway,flavor,looney,comedians,nameberry


In [31]:
top_words_df.iloc[:10,16:]

,IMPACT,ENTERTAINMENT,ARTS & CULTURE,GOOD NEWS,ENVIRONMENT,BUSINESS,WOMEN,COLLEGE,RELIGION,CRIME,FIFTY,LATINO VOICES,MEDIA,SCIENCE,MONEY,WEIRD NEWS,EDUCATION
0,homelessness,trailer,imageblog,pup,networx,krugman,succinct,fraternity,bishops,fatally,boomer,latina,gawker,nasa,credit,fark,charter
1,homeless,teaser,exhibition,pups,sands,investors,feminism,frat,francis,suspect,boomers,latino,journalism,spacecraft,prepaid,ufo,classroom
2,charities,thrones,paintings,pit,tornado,marketing,feminist,colleges,pope,shooter,midlife,latinos,msnbc,astronomers,lottery,squirrel,teachers
3,aspen,swift,nighter,bull,animal,jpmorgan,brighten,universities,interfaith,gunman,retirement,latinas,newsroom,rover,savings,snake,education
4,nonprofits,grande,theatre,kittens,tar,managers,steinem,faculty,prayers,tsarnaev,aging,ferrera,advertisers,telescope,mortgage,pooch,educators
5,charitable,album,opera,prosthetic,sloth,startup,funniest,campus,bishop,manhunt,menopause,shakira,journalists,planets,refund,kangaroo,repayment
6,maternal,reynolds,ballet,paralyzed,spill,startups,metoo,grads,jesus,stabbing,grandchildren,afro,reilly,asteroid,expenses,burglary,educational
7,trafficking,khloe,canvas,bulldog,keystone,tesla,sexism,campuses,catholic,stabbed,grandkids,hispanic,hannity,astronauts,jackpot,alligator,devos
8,nonprofit,ariana,portraits,amputee,panda,marketers,slut,uva,christians,allegedly,retirees,rodriguez,jazeera,mars,finances,hydraulic,educator
9,mutilation,sequel,gallery,selfless,species,entrepreneurial,feminists,professors,maintaining,inmate,retire,despacito,murdoch,astronaut,irs,mph,standardized
